## Day 20
#### Part 1

In [1]:
import numpy as np

In [2]:
# Read in data
input_file = open('day20_input.txt', 'r')
data = input_file.read()[:-2]

In [3]:
# Data wrangling
data = data.split("\n\n")
data = [tile.split("\n") for tile in data]
data = {int(tile[0][5:9]): tile[1:] for tile in data}
data = {k: np.array([list(line) for line in v]) for k, v in data.items()}

In [4]:
def get_tile_flips(arr):
    rotated = [arr, np.rot90(arr, 1), np.rot90(arr, 2), np.rot90(arr, 3)]
    flipped = [np.fliplr(arr) for arr in rotated]
    return rotated + flipped


def get_borders(arr):
    return np.array([arr[0, :], arr[:, 0], arr[len(arr)-1, :], arr[:, len(arr)-1]])


def get_flipped_borders(arr):
    all_borders = []
    for tile in get_tile_flips(arr):
        all_borders.extend(get_borders(tile))
        
    return(np.array(all_borders))

In [5]:
borders = {k: get_borders(v) for k, v in data.items()}
flipped_borders = {k: get_flipped_borders(v) for k, v in data.items()}

In [6]:
matching_borders_dict = {}
for tile, tile_borders in flipped_borders.items():
    matching_borders = []
    tmp_borders = borders.copy()
    tmp_borders.pop(tile)
    for other_tile, other_tile_borders in tmp_borders.items():
        for border in tile_borders:
            if any(np.array_equal(border, other_border) for other_border in other_tile_borders):
                matching_borders.append(other_tile)
    matching_borders_dict[tile] = list(set(matching_borders))

In [7]:
part1_ans = 1
for tile, matches in matching_borders_dict.items():
    if len(matches) == 2:
        part1_ans *= tile
        
print(part1_ans)

111936085519519


#### Part 2

In [8]:
# Get tile map
s = int(np.sqrt(len(data.keys())))
tile_map = np.zeros((s, s)).astype(int)
tile_map[0, 0] = 2797
tile_map[1, 0] = 1291
tile_map[0, 1] = 2719
i = 0
j = 1
tile = 2719
used_tiles = [2797, 1291, 2719]
for j in range(1,11):
    connections = [x for x in matching_borders_dict[tile] if x not in used_tiles]
    for connection in connections:
        if len(matching_borders_dict[connection]) in [2, 3]:
            tile_map[i, j+1] = connection
            used_tiles.append(connection)
            tile = connection            
        else:
            tile_map[i+1, j] = connection
            used_tiles.append(connection)

connection = [x for x in matching_borders_dict[tile] if x not in used_tiles][0]
tile_map[1,11] = connection
used_tiles.append(connection)

for i in range(1,11):
    for j in range(12):
        tile = tile_map[i, j]
        connection = [x for x in matching_borders_dict[tile] if x not in used_tiles][0]
        tile_map[i+1, j] = connection
        used_tiles.append(connection)

In [9]:
def orient_tile(tile, top=None, left=None, right=None, bottom=None):
    tile_flips = get_tile_flips(data[tile])
    if top:     
        tile_flips = [tile_flip for tile_flip in tile_flips
                      if any(map(lambda x: np.array_equal(tile_flip[0, :], x), get_flipped_borders(data[top])))]
    if left:
        tile_flips = [tile_flip for tile_flip in tile_flips
                      if any(map(lambda x: np.array_equal(tile_flip[:, 0], x), get_flipped_borders(data[left])))]
    if right:
        tile_flips = [tile_flip for tile_flip in tile_flips
                      if any(map(lambda x: np.array_equal(tile_flip[:, 9], x), get_flipped_borders(data[right])))]
    if bottom:
        tile_flips = [tile_flip for tile_flip in tile_flips
                      if any(map(lambda x: np.array_equal(tile_flip[9, :], x), get_flipped_borders(data[bottom])))]
        
    return tile_flips

In [10]:
n = 8
image = np.full((s * n, s * n), '0')
for i in range(12):
    for j in range(12):
        tile = tile_map[i, j]
        if i == 0:
            top = None
            bottom = tile_map[i+1, j]
        elif i == 11:
            top = tile_map[i-1, j]
            bottom = None
        else:
            top = tile_map[i-1, j]
            bottom = tile_map[i+1, j]
        if j == 0:
            left = None
            right = tile_map[i, j+1]
        elif j == 11:
            left = tile_map[i, j-1]
            right = None
        else:
            left = tile_map[i, j-1]
            right = tile_map[i, j+1]
        image[i*n:(i+1)*n,j*n:(j+1)*n] = orient_tile(tile, top, left, right, bottom)[0][1:9, 1:9]

In [11]:
sea_monster =  [
"                  # ", 
"#    ##    ##    ###",
" #  #  #  #  #  #   "
]

sea_monster = np.array([np.array(list(line)) for line in sea_monster])
sea_monster_coords = np.where(sea_monster)
sea_monster_coords = list(zip(sea_monster_coords[0], sea_monster_coords[1]))

In [12]:
images = get_tile_flips(image)

for image in images:
    n_sea_monster = 0
    for row in np.arange(len(image) - 3):
        for col in range(len(image) - 20):
            if all([image[x[0]+row, x[1]+col] == '#' for x in sea_monster_coords]):
                n_sea_monster += 1
    if n_sea_monster > 0:
        total = np.count_nonzero(image == '#') - (len(sea_monster_coords) * n_sea_monster)
        print(total)

1792
